Pre requisit: 
- pip install --upgrade mysql-connector-python
- pip install google-api-python-client
- pip install protobuf==3.20.3
- pip install pandas-gbq

In [1]:
from googleapiclient.discovery import build
import pandas as pd
from IPython.display import display, JSON
import re
import json

In [2]:
api_key = "Add Your API KEy"

In [3]:
channel_ids= ['UCJQJAI7IjbLcpsjWdSzYz0Q','UC-PRoQt10P0cnO72j_SxnEQ']

In [4]:
api_service_name = "youtube"
api_version = "v3"

# Get credentials and create an API client
youtube = build(
    api_service_name, api_version, developerKey = api_key)


In [5]:
def get_channel_stats(youtube, channel_ids):
    all_data = []
    request = youtube.channels().list(
                part="snippet,contentDetails,statistics",
                id = ",".join(channel_ids)
    )
    response = request.execute()
    #loop through items
    for item in response['items']:
        data = {'channelName': item['snippet']['title'],
                'total_subscribes': item['statistics']['subscriberCount'],
                'totalviews': item['statistics']['viewCount'],
                'totalVideos': item['statistics']['videoCount'],
                'playlistId': item['contentDetails']['relatedPlaylists']['uploads'],
                'total_video_Count': item['statistics']['videoCount']
               }
        all_data.append(data)
    return(pd.DataFrame(all_data))

In [6]:
channel_stats = get_channel_stats(youtube, channel_ids)

In [7]:
channel_stats

,channelName,total_subscribes,totalviews,totalVideos,playlistId,total_video_Count
0,Thu Vu data analytics,168000,5471413,74,UUJQJAI7IjbLcpsjWdSzYz0Q,74
1,The BI Hub,1230,42643,17,UU-PRoQt10P0cnO72j_SxnEQ,17


### Getting the Video Details of specific Channels

In [8]:
def get_video_ids(youtube, playlist_id):
    video_ids = []
    
    request = youtube.playlistItems().list(
        part="snippet, contentDetails",
        playlistId=playlist_id,
        maxResults = 50
        
    )
    response = request.execute()

    for item in response['items']:
        video_ids.append(item['contentDetails']['videoId'])
    
    next_page_token = response.get('nextPageToken')
    while next_page_token is not None:
        request = youtube.playlistItems().list(
            part="contentDetails",
            playlistId=playlist_id,
            maxResults = 50,
            pageToken = next_page_token
        
    )
        response = request.execute()

        for item in response['items']:
            video_ids.append(item['contentDetails']['videoId'])
    
        next_page_token = response.get('nextPageToken')
        

    return video_ids


In [9]:
for index, row in channel_stats.iterrows():
    playlist_id = row['playlistId']
    print(f"Fetching video IDs for channel: {row['channelName']}")
    video_ids = get_video_ids(youtube, playlist_id)
    print(f"Number of video IDs fetched: {len(video_ids)}")
    print(f"Video IDs: {video_ids}")
    print("----")

Fetching video IDs for channel: Thu Vu data analytics
Number of video IDs fetched: 74
Video IDs: ['EzVIkHQWnhc', '8sLq4P-QDL0', '3736Pd_hwFQ', 'PvnSSrnjLX8', 'S5U76LPu_bQ', 'l2YU8QuXiTM', 'dYQfgcd0XD4', '0JDsvZHooNU', 'kHdAb8yBBDI', 'tSiS15ubQFQ', 'c3Uvz75kVd4', 'aYorBPF05Rw', 'N1UMycRJbAw', '7yGolPv8AOs', 'EE1Y2enHrcU', 'K2wffRzTvBM', 'JPHS10dt_CY', 'hucuMCZBbIY', 'tpGawyNMRLM', '_8EV4tdJY2M', 'QnGotm29cZE', 'ZfN8nG0luig', 'GM8nrVBFdFo', 'GYxiuFRkS58', 'UmFwybBiDt0', 'N0o-Bjiwt0M', 'JsZiFysJS1U', 'uFTd2b23GvI', 'ho9vNL4MYZ8', 'hFLp_aP8iQQ', 'fClT4PWlGoA', 'p5MwJuZZYYY', 'B11nK1QCPRA', 'O_7f4MGeCKE', '5gf2ntjVGe8', 'fAHkJ_Dhr50', 'OIj_uTLplPA', 'RuNolAh_4bU', 'lpSw_RAPmgc', 'UiG8q67Z2XU', 'A8Abf3u0ZIs', 'OEVNDRZxgz8', 'lkMh1g41drk', 'qdzuq_o8-dI', 'eiZbEcMX3b4', 'uhxiXOTKzfs', 'nNfco7k7Hi0', 'MsdyvI3ZI5k', 'U2v76H_B1rs', '8PYJFSGlmmQ', 'D56_Cx36oGY', 'zai2pLUD9FA', 'ir2uJg4DF4k', 'XYKuslcJp7A', 'qPxPoRNIUZk', 'q3gWZkQ4PjU', '0srU1L8wL0U', 'qjLHX3RCayI', 'Kq5iPtAc_3I', 'RM43xgiJhjQ', 'U

In [10]:
def get_video_details(youtube, channel_ids):
    all_video_info = []
    for index, row in channel_stats.iterrows():
        playlist_id = row['playlistId']
        video_ids = get_video_ids(youtube, playlist_id)

        for i in range(0, len(video_ids), 50):
            request = youtube.videos().list(
                part="snippet,contentDetails,statistics",
                id=','.join(video_ids[i:i + 50])
            )
            response = request.execute()

            for video in response['items']:
                stats_to_keep = {
                    'snippet': ['channelTitle', 'title', 'description', 'tags', 'publishedAt'],
                    'statistics': ['viewCount', 'likeCount', 'favoriteCount', 'commentCount'],
                    'contentDetails': ['duration', 'definition', 'caption']
                }
                video_info = {'video_id': video['id']}

                for k in stats_to_keep.keys():
                    for v in stats_to_keep[k]:
                        try:
                            if v == 'tags' and video[k][v]:
                                video_info[v] = ','.join(video[k][v])
                            else:
                                video_info[v] = video[k][v]
                        except:
                            video_info[v] = None
                all_video_info.append(video_info)
    return pd.DataFrame(all_video_info)


In [11]:
video_df = get_video_details(youtube, video_ids)
video_df

,video_id,channelTitle,title,description,tags,publishedAt,viewCount,likeCount,favoriteCount,commentCount,duration,definition,caption
0,EzVIkHQWnhc,Thu Vu data analytics,How to Handle Sensitive Data in Data Science (...,🏆 Check out Antigranular competition & win cas...,"data analytics,data science,python,data,tablea...",2023-08-22T00:29:27Z,4671,240,0,27,PT12M19S,hd,false
1,8sLq4P-QDL0,Thu Vu data analytics,I Tried ChatGPT Code Interpreter for Data Task...,I tell ChatGPT Code Interpreter to do my job 😅...,"data analytics,data science,python,data,tablea...",2023-07-15T19:02:17Z,33837,1571,0,50,PT58S,hd,false
2,3736Pd_hwFQ,Thu Vu data analytics,40 Productivity Tips that Make You More Pro in...,📩 Interested in hearing about my coming course...,"data analytics,data science,python,data,tablea...",2023-07-08T09:45:30Z,12044,620,0,59,PT1H16M9S,hd,false
3,PvnSSrnjLX8,Thu Vu data analytics,Self-taught vs. Guided Program: CareerFoundry'...,📊 Enroll with this link to get 10% off the Dat...,"data analytics,data science,python,data,tablea...",2023-06-16T22:57:31Z,6995,212,0,32,PT16M32S,hd,false
4,S5U76LPu_bQ,Thu Vu data analytics,Is a Computer Science Degree WORTHLESS 🤯? (in ...,I've been thinking lately if it's still worth ...,"data analytics,data science,python,data,tablea...",2023-06-08T16:10:11Z,17805,639,0,58,PT11M8S,hd,false
...,...,...,...,...,...,...,...,...,...,...,...,...,...
86,29AJ9x6xSXw,The BI Hub,How to restrict a slicer to filter out certain...,Edit interactions in power bi\nHow to make a s...,"SQL,Python,Power BI,Data Engineer,Data Analyst...",2023-06-06T08:25:31Z,333,25,0,0,PT1M23S,hd,false
87,SxFp8NJpjz0,The BI Hub,Custom sort order | Sort by column in Power BI,How to custom sort in power bi\nHow to sort co...,"SQL,Python,Power BI,Data Engineer,Data Analyst...",2023-06-04T07:22:50Z,678,44,0,0,PT2M6S,hd,false
88,um0dTXZhuek,The BI Hub,How to set default slicer value in Power BI,How to set latest month as default value in sl...,"SQL,Python,Power BI,Data Engineer,Data Analyst...",2023-06-02T16:11:55Z,5878,125,0,10,PT7M,hd,false
89,sDgYKdPk_iI,The BI Hub,Date table in Power BI,How to create date table in power bi\nHow to c...,"SQL,Data Engineer,Data Analyst,Data Engineerin...",2023-05-31T15:01:23Z,484,26,0,0,PT5M10S,hd,false


### Getting the Videos Comments

In [12]:
def get_comments_in_videos(youtube, channel_ids):
    all_comments = []
    for index, row in channel_stats.iterrows():
        playlist_id = row['playlistId']
        video_ids = get_video_ids(youtube, playlist_id)
        for video_id in video_ids:
            request = youtube.commentThreads().list(
                part="snippet,replies",
                videoId = video_id
            )
            response = request.execute()

            comments_in_video = [comment['snippet']['topLevelComment']['snippet']['textOriginal'] for comment in response['items']]
            comments_as_string = ', '.join(comments_in_video)  # Join comments into a single string
            comments_in_video_info = {'video_id': video_id, 'comments': comments_as_string}
            all_comments.append(comments_in_video_info)

    return pd.DataFrame(all_comments)


In [13]:
comments_df = get_comments_in_videos(youtube, video_ids)
comments_df

,video_id,comments
0,EzVIkHQWnhc,Mam as a ms Excel export please clear my doubt...
1,8sLq4P-QDL0,I think 99% of us have no clue what the releas...
2,3736Pd_hwFQ,What is your favourite tip and trick using dat...
3,PvnSSrnjLX8,My previous upload was cut off half way due to...
4,S5U76LPu_bQ,even without ai cs is useless lol. study in a ...
...,...,...
86,29AJ9x6xSXw,
87,SxFp8NJpjz0,
88,um0dTXZhuek,Can we add this on the basis of Calendar table...
89,sDgYKdPk_iI,


In [14]:
merged_df = pd.merge(channel_stats, video_df, left_on='channelName', right_on='channelTitle')
merged_df.drop(columns=['channelTitle'], inplace=True)
#print(merged_df)
merged_df

,channelName,total_subscribes,totalviews,totalVideos,playlistId,total_video_Count,video_id,title,description,tags,publishedAt,viewCount,likeCount,favoriteCount,commentCount,duration,definition,caption
0,Thu Vu data analytics,168000,5471413,74,UUJQJAI7IjbLcpsjWdSzYz0Q,74,EzVIkHQWnhc,How to Handle Sensitive Data in Data Science (...,🏆 Check out Antigranular competition & win cas...,"data analytics,data science,python,data,tablea...",2023-08-22T00:29:27Z,4671,240,0,27,PT12M19S,hd,false
1,Thu Vu data analytics,168000,5471413,74,UUJQJAI7IjbLcpsjWdSzYz0Q,74,8sLq4P-QDL0,I Tried ChatGPT Code Interpreter for Data Task...,I tell ChatGPT Code Interpreter to do my job 😅...,"data analytics,data science,python,data,tablea...",2023-07-15T19:02:17Z,33837,1571,0,50,PT58S,hd,false
2,Thu Vu data analytics,168000,5471413,74,UUJQJAI7IjbLcpsjWdSzYz0Q,74,3736Pd_hwFQ,40 Productivity Tips that Make You More Pro in...,📩 Interested in hearing about my coming course...,"data analytics,data science,python,data,tablea...",2023-07-08T09:45:30Z,12044,620,0,59,PT1H16M9S,hd,false
3,Thu Vu data analytics,168000,5471413,74,UUJQJAI7IjbLcpsjWdSzYz0Q,74,PvnSSrnjLX8,Self-taught vs. Guided Program: CareerFoundry'...,📊 Enroll with this link to get 10% off the Dat...,"data analytics,data science,python,data,tablea...",2023-06-16T22:57:31Z,6995,212,0,32,PT16M32S,hd,false
4,Thu Vu data analytics,168000,5471413,74,UUJQJAI7IjbLcpsjWdSzYz0Q,74,S5U76LPu_bQ,Is a Computer Science Degree WORTHLESS 🤯? (in ...,I've been thinking lately if it's still worth ...,"data analytics,data science,python,data,tablea...",2023-06-08T16:10:11Z,17805,639,0,58,PT11M8S,hd,false
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
86,The BI Hub,1230,42643,17,UU-PRoQt10P0cnO72j_SxnEQ,17,29AJ9x6xSXw,How to restrict a slicer to filter out certain...,Edit interactions in power bi\nHow to make a s...,"SQL,Python,Power BI,Data Engineer,Data Analyst...",2023-06-06T08:25:31Z,333,25,0,0,PT1M23S,hd,false
87,The BI Hub,1230,42643,17,UU-PRoQt10P0cnO72j_SxnEQ,17,SxFp8NJpjz0,Custom sort order | Sort by column in Power BI,How to custom sort in power bi\nHow to sort co...,"SQL,Python,Power BI,Data Engineer,Data Analyst...",2023-06-04T07:22:50Z,678,44,0,0,PT2M6S,hd,false
88,The BI Hub,1230,42643,17,UU-PRoQt10P0cnO72j_SxnEQ,17,um0dTXZhuek,How to set default slicer value in Power BI,How to set latest month as default value in sl...,"SQL,Python,Power BI,Data Engineer,Data Analyst...",2023-06-02T16:11:55Z,5878,125,0,10,PT7M,hd,false
89,The BI Hub,1230,42643,17,UU-PRoQt10P0cnO72j_SxnEQ,17,sDgYKdPk_iI,Date table in Power BI,How to create date table in power bi\nHow to c...,"SQL,Data Engineer,Data Analyst,Data Engineerin...",2023-05-31T15:01:23Z,484,26,0,0,PT5M10S,hd,false


In [15]:
complete_merged_df = pd.merge(merged_df, comments_df, on='video_id', how = 'left')
complete_merged_df

,channelName,total_subscribes,totalviews,totalVideos,playlistId,total_video_Count,video_id,title,description,tags,publishedAt,viewCount,likeCount,favoriteCount,commentCount,duration,definition,caption,comments
0,Thu Vu data analytics,168000,5471413,74,UUJQJAI7IjbLcpsjWdSzYz0Q,74,EzVIkHQWnhc,How to Handle Sensitive Data in Data Science (...,🏆 Check out Antigranular competition & win cas...,"data analytics,data science,python,data,tablea...",2023-08-22T00:29:27Z,4671,240,0,27,PT12M19S,hd,false,Mam as a ms Excel export please clear my doubt...
1,Thu Vu data analytics,168000,5471413,74,UUJQJAI7IjbLcpsjWdSzYz0Q,74,8sLq4P-QDL0,I Tried ChatGPT Code Interpreter for Data Task...,I tell ChatGPT Code Interpreter to do my job 😅...,"data analytics,data science,python,data,tablea...",2023-07-15T19:02:17Z,33837,1571,0,50,PT58S,hd,false,I think 99% of us have no clue what the releas...
2,Thu Vu data analytics,168000,5471413,74,UUJQJAI7IjbLcpsjWdSzYz0Q,74,3736Pd_hwFQ,40 Productivity Tips that Make You More Pro in...,📩 Interested in hearing about my coming course...,"data analytics,data science,python,data,tablea...",2023-07-08T09:45:30Z,12044,620,0,59,PT1H16M9S,hd,false,What is your favourite tip and trick using dat...
3,Thu Vu data analytics,168000,5471413,74,UUJQJAI7IjbLcpsjWdSzYz0Q,74,PvnSSrnjLX8,Self-taught vs. Guided Program: CareerFoundry'...,📊 Enroll with this link to get 10% off the Dat...,"data analytics,data science,python,data,tablea...",2023-06-16T22:57:31Z,6995,212,0,32,PT16M32S,hd,false,My previous upload was cut off half way due to...
4,Thu Vu data analytics,168000,5471413,74,UUJQJAI7IjbLcpsjWdSzYz0Q,74,S5U76LPu_bQ,Is a Computer Science Degree WORTHLESS 🤯? (in ...,I've been thinking lately if it's still worth ...,"data analytics,data science,python,data,tablea...",2023-06-08T16:10:11Z,17805,639,0,58,PT11M8S,hd,false,even without ai cs is useless lol. study in a ...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
86,The BI Hub,1230,42643,17,UU-PRoQt10P0cnO72j_SxnEQ,17,29AJ9x6xSXw,How to restrict a slicer to filter out certain...,Edit interactions in power bi\nHow to make a s...,"SQL,Python,Power BI,Data Engineer,Data Analyst...",2023-06-06T08:25:31Z,333,25,0,0,PT1M23S,hd,false,
87,The BI Hub,1230,42643,17,UU-PRoQt10P0cnO72j_SxnEQ,17,SxFp8NJpjz0,Custom sort order | Sort by column in Power BI,How to custom sort in power bi\nHow to sort co...,"SQL,Python,Power BI,Data Engineer,Data Analyst...",2023-06-04T07:22:50Z,678,44,0,0,PT2M6S,hd,false,
88,The BI Hub,1230,42643,17,UU-PRoQt10P0cnO72j_SxnEQ,17,um0dTXZhuek,How to set default slicer value in Power BI,How to set latest month as default value in sl...,"SQL,Python,Power BI,Data Engineer,Data Analyst...",2023-06-02T16:11:55Z,5878,125,0,10,PT7M,hd,false,Can we add this on the basis of Calendar table...
89,The BI Hub,1230,42643,17,UU-PRoQt10P0cnO72j_SxnEQ,17,sDgYKdPk_iI,Date table in Power BI,How to create date table in power bi\nHow to c...,"SQL,Data Engineer,Data Analyst,Data Engineerin...",2023-05-31T15:01:23Z,484,26,0,0,PT5M10S,hd,false,


### Feature Engineering

In [16]:
complete_merged_df['publishedAt'] = pd.to_datetime(complete_merged_df['publishedAt'], format='%Y-%m-%dT%H:%M:%SZ')

In [17]:
def convert_duration(duration):
    # Extract minutes and seconds using regular expression
    minutes = re.search(r'(\d+)M', duration)
    seconds = re.search(r'(\d+)S', duration)
    
    # Initialize minutes and seconds as 0 if not present in the duration string
    minutes = int(minutes.group(1)) if minutes else 0
    seconds = int(seconds.group(1)) if seconds else 0
    
    # Return the duration in HH:MM:SS format
    return '{:02}:{:02}:{:02}'.format(0, minutes, seconds)

In [18]:
complete_merged_df['formated_duration'] = complete_merged_df['duration'].apply(convert_duration)
complete_merged_df.drop(columns=['duration'], inplace=True)

In [19]:
# Convert columns to desired data types
complete_merged_df['channelName'] = complete_merged_df['channelName'].astype(str)  # Convert to string
complete_merged_df['total_subscribes'] = complete_merged_df['total_subscribes'].astype(int)  # Convert to integer
complete_merged_df['totalviews'] = complete_merged_df['totalviews'].astype(int)  # Convert to integer
complete_merged_df['totalVideos'] = complete_merged_df['totalVideos'].astype(int)  # Convert to integer
complete_merged_df['playlistId'] = complete_merged_df['playlistId'].astype(str)  # Convert to string
complete_merged_df['total_video_Count'] = complete_merged_df['total_video_Count'].astype(int)  # Convert to integer
complete_merged_df['video_id'] = complete_merged_df['video_id'].astype(str)  # Convert to string
complete_merged_df['title'] = complete_merged_df['title'].astype(str)  # Convert to string
complete_merged_df['description'] = complete_merged_df['description'].astype(str)  # Convert to string
complete_merged_df['tags'] = complete_merged_df['tags'].astype(str)  # Convert to string
complete_merged_df['publishedAt'] = pd.to_datetime(complete_merged_df['publishedAt'])  # Convert to datetime
complete_merged_df['viewCount'] = complete_merged_df['viewCount'].astype(int)  # Convert to integer
complete_merged_df['likeCount'] = complete_merged_df['likeCount'].astype(int)  # Convert to integer
complete_merged_df['favoriteCount'] = complete_merged_df['favoriteCount'].astype(int)  # Convert to integer
complete_merged_df['commentCount'] = complete_merged_df['commentCount'].astype(int)  # Convert to integer
complete_merged_df['definition'] = complete_merged_df['definition'].astype(str)  # Convert to string
complete_merged_df['caption'] = complete_merged_df['caption'].astype(str)  # Convert to string
complete_merged_df['comments'] = complete_merged_df['comments'].astype(str)  # Convert to string



In [20]:
complete_merged_df

,channelName,total_subscribes,totalviews,totalVideos,playlistId,total_video_Count,video_id,title,description,tags,publishedAt,viewCount,likeCount,favoriteCount,commentCount,definition,caption,comments,formated_duration
0,Thu Vu data analytics,168000,5471413,74,UUJQJAI7IjbLcpsjWdSzYz0Q,74,EzVIkHQWnhc,How to Handle Sensitive Data in Data Science (...,🏆 Check out Antigranular competition & win cas...,"data analytics,data science,python,data,tablea...",2023-08-22 00:29:27,4671,240,0,27,hd,false,Mam as a ms Excel export please clear my doubt...,00:12:19
1,Thu Vu data analytics,168000,5471413,74,UUJQJAI7IjbLcpsjWdSzYz0Q,74,8sLq4P-QDL0,I Tried ChatGPT Code Interpreter for Data Task...,I tell ChatGPT Code Interpreter to do my job 😅...,"data analytics,data science,python,data,tablea...",2023-07-15 19:02:17,33837,1571,0,50,hd,false,I think 99% of us have no clue what the releas...,00:00:58
2,Thu Vu data analytics,168000,5471413,74,UUJQJAI7IjbLcpsjWdSzYz0Q,74,3736Pd_hwFQ,40 Productivity Tips that Make You More Pro in...,📩 Interested in hearing about my coming course...,"data analytics,data science,python,data,tablea...",2023-07-08 09:45:30,12044,620,0,59,hd,false,What is your favourite tip and trick using dat...,00:16:09
3,Thu Vu data analytics,168000,5471413,74,UUJQJAI7IjbLcpsjWdSzYz0Q,74,PvnSSrnjLX8,Self-taught vs. Guided Program: CareerFoundry'...,📊 Enroll with this link to get 10% off the Dat...,"data analytics,data science,python,data,tablea...",2023-06-16 22:57:31,6995,212,0,32,hd,false,My previous upload was cut off half way due to...,00:16:32
4,Thu Vu data analytics,168000,5471413,74,UUJQJAI7IjbLcpsjWdSzYz0Q,74,S5U76LPu_bQ,Is a Computer Science Degree WORTHLESS 🤯? (in ...,I've been thinking lately if it's still worth ...,"data analytics,data science,python,data,tablea...",2023-06-08 16:10:11,17805,639,0,58,hd,false,even without ai cs is useless lol. study in a ...,00:11:08
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
86,The BI Hub,1230,42643,17,UU-PRoQt10P0cnO72j_SxnEQ,17,29AJ9x6xSXw,How to restrict a slicer to filter out certain...,Edit interactions in power bi\nHow to make a s...,"SQL,Python,Power BI,Data Engineer,Data Analyst...",2023-06-06 08:25:31,333,25,0,0,hd,false,,00:01:23
87,The BI Hub,1230,42643,17,UU-PRoQt10P0cnO72j_SxnEQ,17,SxFp8NJpjz0,Custom sort order | Sort by column in Power BI,How to custom sort in power bi\nHow to sort co...,"SQL,Python,Power BI,Data Engineer,Data Analyst...",2023-06-04 07:22:50,678,44,0,0,hd,false,,00:02:06
88,The BI Hub,1230,42643,17,UU-PRoQt10P0cnO72j_SxnEQ,17,um0dTXZhuek,How to set default slicer value in Power BI,How to set latest month as default value in sl...,"SQL,Python,Power BI,Data Engineer,Data Analyst...",2023-06-02 16:11:55,5878,125,0,10,hd,false,Can we add this on the basis of Calendar table...,00:07:00
89,The BI Hub,1230,42643,17,UU-PRoQt10P0cnO72j_SxnEQ,17,sDgYKdPk_iI,Date table in Power BI,How to create date table in power bi\nHow to c...,"SQL,Data Engineer,Data Analyst,Data Engineerin...",2023-05-31 15:01:23,484,26,0,0,hd,false,,00:05:10


### Adding the data frame to the BigQuery

In [21]:
from pandas_gbq import to_gbq

# Define your BigQuery project ID, dataset ID, and table name
project_id = "youtube-dataapi-400711"
dataset_id = "youtube-dataapi-400711.ChannelStatistics"
table_name = "ChannelStatistics"

# Upload the DataFrame to BigQuery
destination_table = f"{dataset_id}.{table_name}"
to_gbq(complete_merged_df, destination_table=destination_table, project_id=project_id, if_exists="replace")

print(f"DataFrame uploaded to BigQuery: {project_id}.{destination_table}")

100%|████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<?, ?it/s]

DataFrame uploaded to BigQuery: youtube-dataapi-400711.youtube-dataapi-400711.ChannelStatistics.ChannelStatistics


In [22]:
# excel_file_path = 'C:/Users/Krishan Singh/Desktop/data.xlsx'
# video_df.to_excel(excel_file_path, index=False)
# print(f'DataFrame has been exported to {excel_file_path}')

Now Using the data we can create charts and reports.